## 如何调试梯度

In [1]:
import numpy as np

In [2]:
np.random.seed(666)
X = np.random.random(size=(1000, 10))

In [3]:
X.shape

(1000, 10)

In [4]:
true_theta = np.arange(1, 12, dtype=float)
true_theta

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [5]:
X_b = np.hstack([np.ones((len(X), 1)), X])
y = X_b.dot(true_theta) + np.random.normal(size=len(X_b)) * 1.7
y.shape

(1000,)

In [6]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta))**2) / len(X_b)
    except:
        return float('inf')

In [7]:
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

In [8]:
def dJ_debug(theta, X_b, y, epsilon=0.01):
    result = np.empty(len(theta))
    
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        result[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2 * epsilon) 
    
    return result

In [9]:
def gradient_descent(dJ, X_b, y, inti_theta, eta=0.01, n_iters=1e4, epsilon=1e-8):
    
    i_iters = 0
    theta = inti_theta
    
    while i_iters < n_iters:
        last_theta = theta
        
        gradient = dJ(theta, X_b, y)
        theta = last_theta - eta * gradient
        
        if abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon:
            break;
        i_iters += 1
        
    print("iters =", i_iters)
    return theta

In [10]:
X_b = np.hstack([np.ones((len(X), 1)), X])
init_theta = np.zeros(X_b.shape[1])

In [11]:
%time theta = gradient_descent(dJ_debug, X_b, y, init_theta)
theta

iters = 10000
CPU times: user 3.16 s, sys: 8.95 ms, total: 3.17 s
Wall time: 3.17 s


array([ 1.16848078,  2.09615138,  2.86563275,  4.21266024,  5.09364724,
        5.84595206,  6.96520772,  8.00857726,  8.77585158,  9.98413583,
       10.84841499])

In [12]:
%time theta = gradient_descent(dJ_math, X_b, y, init_theta)
theta

iters = 10000
CPU times: user 3.72 s, sys: 33.8 ms, total: 3.76 s
Wall time: 474 ms


array([ 1.16848078,  2.09615138,  2.86563275,  4.21266024,  5.09364724,
        5.84595206,  6.96520772,  8.00857726,  8.77585158,  9.98413583,
       10.84841499])

## 从上面的结果看，两种求梯度的方式，最后的到的数据结果是差不多的，说明 debug 方式是可用的。

## 但是 debug 模式的耗时比 math 模式要多得多，因此 debug 模式只能用在小数据量的情况及用来验证 meth 模式的数据是否可靠。

## debug 模式还有一个优点：dJ_math 是和具体的损失函数的求(偏)导数公式有关，不同的损失函数会推导出不同的 dJ_math 内容， 而 dJ_debug 和 导数无关，只和 损失函数本身有关，只要 J 函数定义好了，debug 模式中的代码是一致的，所以可以包装成一个工具类。